In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import torch
%matplotlib inline

import time
import numpy as np
from torchvision import datasets
from torchvision import transforms
from torch.utils.data import DataLoader
import torch.nn.functional as F
import torch

#########################
### SETTINGS
##########################

RANDOM_SEED = 1
BATCH_SIZE = 100
NUM_EPOCHS = 10
DEVICE = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')


##########################
### MNIST DATASET
##########################
# Note transforms.ToTensor() scales input images
# to 0-1 range
train_dataset = datasets.MNIST(root='data', 
                               train=True, 
                               transform=transforms.ToTensor(),
                               download=True)

test_dataset = datasets.MNIST(root='data', 
                              train=False, 
                              transform=transforms.ToTensor())


train_loader = DataLoader(dataset=train_dataset, 
                          batch_size=BATCH_SIZE, 
                          shuffle=True)

test_loader = DataLoader(dataset=test_dataset, 
                         batch_size=BATCH_SIZE, 
                         shuffle=False)

# Checking the dataset
for images, labels in train_loader:  
    print('Image batch dimensions:', images.shape)
    print('Image label dimensions:', labels.shape)
    break

Image batch dimensions: torch.Size([100, 1, 28, 28])
Image label dimensions: torch.Size([100])


In [2]:
def to_onehot(y, num_classes):
    y_onehot = torch.FloatTensor(y.size(0), num_classes)
    y_onehot.zero_()
    tmp = y.view(-1, 1).long().to(torch.device('cpu'))
    y_onehot.scatter_(1, tmp, 1).float()
    return y_onehot

### 添加scheduler 需要如下几步
- from torch.optim.lr_scheduler import MultiStepLR

- 定义scheduler
scheduler = MultiStepLR(optimizer, [2, 4,6,8], 0.1)

- 在循环中添加如下代码
for epoch in range(NUM_EPOCHS):
    scheduler.step()
    print(epoch, 'lr={:.6f}'.format(scheduler.get_last_lr()[0]))

In [3]:
from torch.optim.lr_scheduler import MultiStepLR
NUM_EPOCHS = 10

class MlpSigmoidMSE(torch.nn.Module):

    def __init__(self, num_features, num_hidden, num_classes):
        super(MlpSigmoidMSE, self).__init__()
        
        self.num_classes = num_classes
        
        ### 1st hidden layer
        self.linear_1 = torch.nn.Linear(num_features, num_hidden)
        self.linear_1.weight.detach().normal_(0.0, 0.1)
        self.linear_1.bias.detach().zero_()

        ### Output layer
        self.linear_out = torch.nn.Linear(num_hidden, num_classes)
        self.linear_out.weight.detach().normal_(0.0, 0.1)
        self.linear_out.bias.detach().zero_()
        
    def forward(self, x):
        out = self.linear_1(x)
        out = torch.sigmoid(out)
        logits = self.linear_out(out)
        probas = torch.sigmoid(logits)
        return logits, probas

    
#################################
### Model Initialization
#################################
    
torch.manual_seed(RANDOM_SEED)
model = MlpSigmoidMSE(num_features=28*28,
                      num_hidden=100,
                      num_classes=10)

model = model.to(DEVICE)

optimizer = torch.optim.SGD(model.parameters(), lr=0.1)
scheduler = MultiStepLR(optimizer, [2, 4,6,8], 0.1)
#################################
### Training
#################################

def compute_mse(net, data_loader):
    curr_mse, num_examples = torch.zeros(model.num_classes).float(), 0
    with torch.no_grad():
        for features, targets in data_loader:
            features = features.view(-1, 28*28).to(DEVICE)
            logits, probas = net.forward(features)
            probas = probas.to(torch.device('cpu'))
            y_onehot = to_onehot(targets, model.num_classes)
            loss = torch.sum((y_onehot - probas)**2, dim=0)
            num_examples += targets.size(0)
            curr_mse += loss

        curr_mse = torch.mean(curr_mse/num_examples, dim=0)
        return curr_mse



start_time = time.time()
minibatch_cost = []
epoch_cost = []
for epoch in range(NUM_EPOCHS):
    scheduler.step()
    print(epoch, 'lr={:.6f}'.format(scheduler.get_last_lr()[0]))
    model.train()
    for batch_idx, (features, targets) in enumerate(train_loader):
        
        features = features.view(-1, 28*28).to(DEVICE)
            
        ### FORWARD AND BACK PROP
        logits, probas = model(features)
        
        y_onehot = to_onehot(targets, model.num_classes).to(DEVICE)
        
        cost = F.mse_loss(y_onehot, probas)
        optimizer.zero_grad()
        
        cost.backward()
        minibatch_cost.append(cost.item())
        ### UPDATE MODEL PARAMETERS
        optimizer.step()
        
        ### LOGGING
        if not batch_idx % 50:
            print ('Epoch: %03d/%03d | Batch %03d/%03d | Cost: %.4f' 
                   %(epoch+1, NUM_EPOCHS, batch_idx, 
                     len(train_loader), cost.item()))
        
    cost = compute_mse(model, train_loader)
    epoch_cost.append(cost.item())
    print('Epoch: %03d/%03d Train Cost: %.4f' % (
            epoch+1, NUM_EPOCHS, cost.item()))
    print('Time elapsed: %.2f min' % ((time.time() - start_time)/60))
    
print('Total Training Time: %.2f min' % ((time.time() - start_time)/60))

D:\ProgramData\Anaconda3\lib\site-packages\torch\optim\lr_scheduler.py:129: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn("Detected call of `lr_scheduler.step()` before `optimizer.step()`. "


0 lr=0.100000
Epoch: 001/010 | Batch 000/600 | Cost: 0.2527
Epoch: 001/010 | Batch 050/600 | Cost: 0.1039
Epoch: 001/010 | Batch 100/600 | Cost: 0.0944
Epoch: 001/010 | Batch 150/600 | Cost: 0.0912
Epoch: 001/010 | Batch 200/600 | Cost: 0.0907
Epoch: 001/010 | Batch 250/600 | Cost: 0.0899
Epoch: 001/010 | Batch 300/600 | Cost: 0.0897
Epoch: 001/010 | Batch 350/600 | Cost: 0.0890
Epoch: 001/010 | Batch 400/600 | Cost: 0.0890
Epoch: 001/010 | Batch 450/600 | Cost: 0.0884
Epoch: 001/010 | Batch 500/600 | Cost: 0.0880
Epoch: 001/010 | Batch 550/600 | Cost: 0.0887
Epoch: 001/010 Train Cost: 0.0881
Time elapsed: 0.14 min
1 lr=0.010000
Epoch: 002/010 | Batch 000/600 | Cost: 0.0878
Epoch: 002/010 | Batch 050/600 | Cost: 0.0880
Epoch: 002/010 | Batch 100/600 | Cost: 0.0877
Epoch: 002/010 | Batch 150/600 | Cost: 0.0871
Epoch: 002/010 | Batch 200/600 | Cost: 0.0873
Epoch: 002/010 | Batch 250/600 | Cost: 0.0876
Epoch: 002/010 | Batch 300/600 | Cost: 0.0872
Epoch: 002/010 | Batch 350/600 | Cost: 0.